Adding single abilities to Spuds

In [ ]:
using Random
using DataFrames
using CSV
using DelimitedFiles
using Statistics
using Printf
using LinearAlgebra
using Dates

In [ ]:
struct Spud
    name::String
    h::Int64
    f::Int64
    l::Int64
    p::Int64
    r::Int64
    s::Int64
    a1::Int64
end


const idx_your_h = 1
const idx_your_f = 2
const idx_your_l = 3
const idx_your_p = 4
const idx_your_r = 5
const idx_your_s = 6
const idx_opp_h = 7
const idx_opp_f = 8
const idx_opp_l = 9
const idx_opp_p = 10
const idx_opp_r = 11
const idx_opp_s = 12

const ab_cond_min = 1
const ab_cond_max = 2
const ab_cond_number_min_opp = 3 # number of min stats for opponent is greater than
const ab_cond_number_max_opp = 4 # number of max stats for opponent is greater than
const ab_cond_diff_min = 5 # difference between stat1 and stat2 is at least 
const ab_cond_ratio_min = 6 # ratio of stat1 and stat2 at least

const ab_fx_modstat = 1 # modify stat1
const ab_fx_copystat = 2 # copy value of stat2 to stat1
const ab_fx_swapstat = 3 # switch values of stat2 and stat2

# r_h required h
# m_h static h modifier
struct ForcedAbility
    r_h::Int64
    r_f::Int64
    r_l::Int64
    r_p::Int64
    r_r::Int64
    r_s::Int64
    m_h::Int64
    m_f::Int64
    m_l::Int64
    m_p::Int64
    m_r::Int64
    m_s::Int64
    cond_type::Int64
    cond_statidx1::Int64
    cond_statidx2::Int64
    cond_thres1::Int64
    cond_thres2::Int64
    fx_type::Int64
    fx_statidx1::Int64
    fx_statidx2::Int64
    fx_mult::Int64
    fx_add::Int64
end

function c_forcedability(
        ; r_h::Int64 = 0, 
        r_f::Int64 = 0,
        r_l::Int64 = 0,
        r_p::Int64 = 0,
        r_r::Int64 = 0,
        r_s::Int64 = 0,
        m_h::Int64 = 0, 
        m_f::Int64 = 0,
        m_l::Int64 = 0,
        m_p::Int64 = 0,
        m_r::Int64 = 0,
        m_s::Int64 = 0,
        cond_type::Int64 = 0,
        cond_statidx1::Int64 = 0,
        cond_statidx2::Int64 = 0,
        cond_thres1::Int64 = 0,
        cond_thres2::Int64 = 0,
        fx_type::Int64 = 0,
        fx_statidx1::Int64 = 0,
        fx_statidx2::Int64 = 0,
        fx_mult::Int64 = 1,
        fx_add::Int64 = 0,        
    )
    return ForcedAbility(
        r_h, r_f, r_l, r_p, r_r, r_s, 
        m_h, m_f, m_l, m_p, m_r, m_s, 
        cond_type, cond_statidx1, cond_statidx2, cond_thres1, cond_thres2,
        fx_type, fx_statidx1, fx_statidx2, fx_mult, fx_add
    )
end



In [ ]:
const ability_none = 999
const ability_imitative = 10
const ability_reciprocating = 20
const ability_romantic = 30
const ability_bibliophile = 40
const ability_melodious = 50
#const ability_poetic = 60
#const ability_acrobatic = 70
#const ability_critical = 80
const ability_iconoclast = 90
const ability_intoxicating = 100
const ability_otaku = 110
const ability_sincere = 120
const ability_cowardly = 130
const ability_reckless = 140
const ability_taunting = 150
const ability_envious = 160





ability_name = Dict(
    ability_imitative => "Imitative*",
    ability_reciprocating => "Reciprocating*",
    ability_romantic => "Romantic*",
    ability_bibliophile => "Bibliophile*",
    ability_melodious => "Melodious*",
    #ability_poetic => "Poetic*",
    #ability_acrobatic => "Acrobatic*",
    #ability_critical => "Critical*",
    ability_iconoclast => "Iconoclast*",
    ability_intoxicating => "Intoxicating*",
    ability_otaku => "Otaku*",
    ability_sincere => "Sincere*",
    ability_cowardly => "Cowardly*",
    ability_taunting => "Taunting*",
    ability_envious => "Envious*",
)

d_ability_imitative = c_forcedability(
    r_h = 6, m_h = -5, 
    cond_type = ab_cond_diff_min, cond_statidx1 = idx_opp_h, cond_statidx2 = idx_your_h, cond_thres1 = 0, 
    fx_type = ab_fx_copystat, fx_statidx1 = idx_your_h, fx_statidx2 = idx_opp_h
)

ability_data = Dict(
    ability_imitative => d_ability_imitative,
)

In [ ]:
const MXS = 100
const MXV = 10
const MNV = 1

function cost(h::Int64, f::Int64, l::Int64, p::Int64, r::Int64, s::Int64)::Int64
    #return h*(f+l-2*MNV) + f*(p+r+s-3*MNV) + h+f+l+p+r+s-6*MNV
    return h*(f+l) + f*(p+r+s)
end

function cost(a::Spud)::Int64
    return cost(a.h, a.f, a.l, a.p, a.r, a.s)
end

In [ ]:
function compare_int_list(as::Vector{Int64}, bs::Vector{Int64}, tiebreaker::Int64 = 0)::Int64
    n = min(length(as), length(bs))
    for i in 1:n
        if as[i] != bs[i]
            return sign(as[i] - bs[i])
        end
    end
    return tiebreaker
end

function spud_h_seq(a::Spud)::Vector{Int64}
    return [a.h, a.s, a.r, a.p, a.l, a.f]
end

function spud_f_seq(a::Spud)::Vector{Int64}
    return [a.f, a.s, a.r, a.p, a.l, a.h]
end

function spud_l_seq(a::Spud)::Vector{Int64}
    return [a.l]
end

function spud_p_seq(a::Spud)::Vector{Int64}
    return [a.p, a.l]
end

function spud_r_seq(a::Spud)::Vector{Int64}
    return [a.r, a.f]
end

function spud_s_seq(a::Spud)::Vector{Int64}
    return [a.s, a.h]
end

function spud_utb_seq(a::Spud)::Vector{Int64}
    return [a.a1, a.h, a.f, a.l, a.p, a.r, a.s]
end

function eval_finds(a::Spud, b::Spud, tiebreaker::Int64 = 0)::Int64
    ev = compare_int_list(spud_f_seq(a), spud_h_seq(b), tiebreaker)
    ans = ev
end

function eval_melee(a::Spud, b::Spud, tiebreaker1::Int64 = 0, tiebreaker2::Int64 = 0)::Int64
    comp_p = compare_int_list(spud_p_seq(a), spud_p_seq(b), tiebreaker1)
    comp_r = compare_int_list(spud_r_seq(a), spud_r_seq(b), tiebreaker1)
    comp_s = compare_int_list(spud_s_seq(a), spud_s_seq(b), tiebreaker1)
    ev = 4 * comp_p + 3 * comp_r + 2 * comp_s
    return sign(ev + (1-abs(ev))*tiebreaker2)
end



In [ ]:
# const idx_your_h = 1
# const idx_your_f = 2
# const idx_your_l = 3
# const idx_your_p = 4
# const idx_your_r = 5
# const idx_your_s = 6
# const idx_opp_h = 7
# const idx_opp_f = 8
# const idx_opp_l = 9
# const idx_opp_p = 10
# const idx_opp_r = 11
# const idx_opp_s = 12

# const ab_cond_min = 1
# const ab_cond_max = 2
# const ab_cond_number_min_opp = 3 # number of min stats for opponent is greater than
# const ab_cond_number_max_opp = 4 # number of max stats for opponent is greater than
# const ab_cond_diff_min = 5 # difference between stat1 and stat2 is at least 
# const ab_cond_ratio_min = 6 # ratio of stat1 and stat2 at least

# const ab_fx_modstat = 1 # modify stat1
# const ab_fx_copystat = 2 # copy value of stat2 to stat1
# const ab_fx_swapstat = 3 # switch values of stat2 and stat2


# ForcedAbility(
#         r_h, r_f, r_l, r_p, r_r, r_s, 
#         m_h, m_f, m_l, m_p, m_r, m_s, 
#         cond_type, cond_statidx1, cond_statidx2, cond_thres1, cond_thres2,
#         fx_type, fx_statidx1, fx_statidx2, fx_mult, fx_add
#     )

function apply_ability_static(a::Spud)::Spud
    ab = ability_data[a.a1]
    return Spud(
        a.name, 
        a.h + ab.m_h, a.f + ab.m_f, a.l + ab.m_l, 
        a.p + ab.m_p, a.r + ab.m_r, a.s + ab.m_s, a.a1)
end

function get_stat_value(a::Spud, b::Spud, statidx::Int64)::Int64
    svs = [a.h, a.f, a.l, a.p, a.r, a.s, b.h, b.f, b.l, b.p, b.r, b.s]
    return svs[statidx]
end


function set_stat_value0(a::Spud, statidx::Int64, val::Int64)::Spud
    if statidx == 1
        return Spud(a.name, val, a.f, a.l, a.p, a.r, a.s, a.a1)
    end
    if statidx == 2
        return Spud(a.name, a.h, val, a.l, a.p, a.r, a.s, a.a1)
    end
    if statidx == 3
        return Spud(a.name, a.h, a.f, val, a.p, a.r, a.s, a.a1)
    end
    if statidx == 4
        return Spud(a.name, a.h, a.f, a.l, val, a.r, a.s, a.a1)
    end
    if statidx == 5
        return Spud(a.name, a.h, a.f, a.l, a.p, val, a.s, a.a1)
    end
    if statidx == 6
        return Spud(a.name, a.h, a.f, a.l, a.p, a.r, val, a.a1)
    end
end


function set_stat_value(a::Spud, b::Spud, statidx::Int64, val::Int64)::Array{Spud}
    if statidx <= 6
        a = set_stat_value0(a, statidx, val)
    end
    if statidx >= 6
        b = set_stat_value0(b, statidx - 6, val)
    end
    return [a,b]
end


function apply_ability(a::Spud, b::Spud)::Array{Spud}
    ab = ability_data[a.a1]
    if a.a1 == ability_none
        return [a, b]
    end
    # check condition
    cond_pass = false
    if ab.cond_type == ab_cond_min
        cond_pass = (get_stat_value(a, b, ab.cond_statidx1) >= ab.cond_thres1)
    end
    if ab.cond_type == ab_cond_max
        cond_pass = (get_stat_value(a, b, ab.cond_statidx1) <= ab.cond_thres1)
    end
    if ab.cond_type == ab_cond_number_min_opp
        cond_pass = (sum([b.h, b.f, b.l, b.p, b.r, b.s] .>= ab.cond_thres1) >= ab.cond_thres2)
    end
    if ab.cond_type == ab_cond_number_max_opp
        cond_pass = (sum([b.h, b.f, b.l, b.p, b.r, b.s] .<= ab.cond_thres1) >= ab.cond_thres2)
    end
    if ab.cond_type == ab_cond_diff_min
        cond_pass = (get_stat_value(a, b, ab.cond_statidx1) - get_stat_value(a, b, ab.cond_statidx2) >= ab.cond_thres1)
    end
    if ab.cond_type == ab_cond_ratio_min
        cond_pass = (get_stat_value(a, b, ab.cond_statidx1) >= get_stat_value(a, b, ab.cond_statidx2) * ab.cond_thres1)
    end
    if cond_pass
        if ab.fx_type == ab_fx_modstat
            res = set_stat_value(a, b, ab.fx_statidx1, get_stat_value(a, b, ab.fx_statidx1) * ab.fx_mult + ab.fx_add)
        end
        if ab.fx_type == ab_fx_copystat
            res = set_stat_value(a, b, ab.fx_statidx1, get_stat_value(a, b, ab.fx_statidx2))
        end
        if ab.fx_type == ab_fx_swapstat
            temp = get_stat_value(a, b, ab.fx_statidx1)
            res = set_stat_value(a, b, ab.fx_statidx1, get_stat_value(a, b, ab.fx_statidx2))
            res = set_stat_value(a, b, ab.fx_statidx2, temp)
        end
        return res
    else
        return [a,b]
    end
end


function eval_battle(a::Spud, b::Spud)::Int64
    utb = compare_int_list(spud_utb_seq(a), spud_utb_seq(b), 0) # universal tiebreaker
    if utb == 0
        return 0
    end
    if a.a1 != ability_none
        a = apply_ability_static(a)
    end
    if b.a1 != ability_none
        b = apply_ability_static(b)
    end
    a_ability_first = compare_int_list([a.a1, a.s], [b.a1, b.s], utb)
    if a_ability_first ==1
        if a.a1 != ability_none
            res = apply_ability(a, b)
            a = res[1]
            b = res[2]
        end
        if b.a1 != ability_none
            res = apply_ability(b, a)
            b = res[1]
            a = res[2]
        end
    end
    if a_ability_first == -1
        if b.a1 != ability_none
            res = apply_ability(b, a)
            b = res[1]
            a = res[2]
        end
        if a.a1 != ability_none
            res = apply_ability(a, b)
            a = res[1]
            b = res[2]
        end
    end
    #println(a)
    #println(b)
    a_finds = eval_finds(a, b, utb)==1
    b_finds = eval_finds(b, a, utb)==1
    melee_win = eval_melee(a, b, 0, utb)
    if a_finds && b_finds
        return melee_win
    end
    if a_finds && !b_finds
        return 1
    end
    if !a_finds && b_finds
        return -1
    end
    if !a_finds && !b_finds
        return compare_int_list([a.l, melee_win], [b.l, -melee_win], 0)
    end
end

In [ ]:
function meet_ability_req(ff::Spud, ab::ForcedAbility)::Bool
    return (ff.h >= ab.r_h) && (ff.f >= ab.r_f) && (ff.l >= ab.r_l) && (ff.p >= ab.r_p) && (ff.r >= ab.r_r) && (ff.s >= ab.r_s)
end

function meet_ability_req(ff::Spud, ab::Int64)::Bool
    return meet_ability_req(ff, ability_data[ab])
end

In [ ]:
b = Spud("tester", 4, 3, 2, 3, 3, 3, ability_none)
a = Spud("ab_im", 2, 3, 3, 1, 1, 1, ability_imitative)
a0 = Spud("ab_im", 2, 3, 3, 1, 1, 1, ability_none)
a1 = Spud("ab_im", 6, 3, 3, 1, 1, 1, ability_none)


In [ ]:
apply_ability_static(a)

In [ ]:
res = apply_ability(a, b)

In [ ]:
eval_battle(a0, b)

In [ ]:
eval_battle(a1, b)

In [ ]:
eval_battle(a, b)

In [ ]:
tab = CSV.read("census_yob2022_names.txt", DataFrame, header = false)
names = tab.Column1
adjectives = CSV.read("adjectives.csv", DataFrame)
nouns = CSV.read("nouns.csv", DataFrame)
jobs = CSV.read("jobs.csv", DataFrame)


const mult_noun = 1
const mult_adj = 2
const mult_job = 3


function random_name_and_stat(ability::Int64 = ability_none)::Spud
    vp = [0, 0, 0, 0, 0, 0]
    #nametype = rand([1,1,1,1,1,2,2,2,3])
    name = ""
    noun = ""
    adj = ""
    noun_i = rand(1:nrow(nouns))
    noun = nouns[noun_i, :noun]
    vp[1] = vp[1] + nouns[noun_i, :H] * mult_noun
    vp[2] = vp[2] + nouns[noun_i, :F] * mult_noun
    vp[3] = vp[3] + nouns[noun_i, :L] * mult_noun
    vp[4] = vp[4] + nouns[noun_i, :P] * mult_noun
    vp[5] = vp[5] + nouns[noun_i, :R] * mult_noun
    vp[6] = vp[6] + nouns[noun_i, :S] * mult_noun
    adj_i = rand(1:nrow(adjectives))
    if ability == ability_none
        adj = adjectives[adj_i, :adjective]
        vp[1] = vp[1] + adjectives[adj_i, :H] * mult_adj
        vp[2] = vp[2] + adjectives[adj_i, :F] * mult_adj
        vp[3] = vp[3] + adjectives[adj_i, :L] * mult_adj
        vp[4] = vp[4] + adjectives[adj_i, :P] * mult_adj
        vp[5] = vp[5] + adjectives[adj_i, :R] * mult_adj
        vp[6] = vp[6] + adjectives[adj_i, :S] * mult_adj
    else
        adj = ability_name[ability]
    end
    job_i = rand(1:nrow(jobs))
    job = jobs[job_i, :job]
    vp[1] = vp[1] + jobs[job_i, :H] * mult_job
    vp[2] = vp[2] + jobs[job_i, :F] * mult_job
    vp[3] = vp[3] + jobs[job_i, :L] * mult_job
    vp[4] = vp[4] + jobs[job_i, :P] * mult_job
    vp[5] = vp[5] + jobs[job_i, :R] * mult_job
    vp[6] = vp[6] + jobs[job_i, :S] * mult_job
    name = string(adj, " ", job, " ", noun)
    Spud(name, vp[1], vp[2], vp[3], vp[4], vp[5], vp[6], ability_none)
end

In [ ]:
function rand_rename(a::Spud, n_tries::Int = 100)::Spud
    best_score = 0.0
    best_b = random_name_and_stat(a.a1)
    for ii in 1:n_tries
        b = random_name_and_stat(a.a1)
        b_norm = sqrt(b.h^2 + b.f^2 + b.l^2 + b.p^2 + b.r^2 + b.s^2)
        score = (a.h * b.h + a.f * b.f + a.l * b.l + a.p * b.p + a.r * b.r + a.s * b.s)/b_norm
        if score > best_score
            best_score = score
            best_b = b
        end
    end
    return Spud(best_b.name, a.h, a.f, a.l, a.p, a.r, a.s, a.a1)
end

In [ ]:
rand_rename(a)

In [ ]:
function eval_battle_list(a::Spud, bs::Array{Spud})::Int
    score = 0
    for ii in 1:length(bs)
        score = score + eval_battle(a, bs[ii])
    end
    return score
end

function eval_battle_list2(a::Spud, bs::Array{Spud}, w::Vector{Float64})::AbstractFloat
    score = 0.0
    for ii in 1:length(bs)
        score = score + w[ii] * eval_battle(a, bs[ii])
    end
    return score
end

function spuds_to_df(as::Array{Spud})::DataFrame
    names = Array{String}(undef, length(as))
    hs = Array{Int}(undef, length(as))
    fs = Array{Int}(undef, length(as))
    ls = Array{Int}(undef, length(as))
    ps = Array{Int}(undef, length(as))
    rs = Array{Int}(undef, length(as))
    ss = Array{Int}(undef, length(as))
    a1s = Array{Int}(undef, length(as))
    for ii in 1:length(as)
        names[ii] = as[ii].name
        hs[ii] = as[ii].h
        fs[ii] = as[ii].f
        ls[ii] = as[ii].l
        ps[ii] = as[ii].p
        rs[ii] = as[ii].r
        ss[ii] = as[ii].s
        a1s[ii] = as[ii].a1
    end
    df = DataFrame(name = names, h = hs, f = fs, l = ls, p = ps, r = rs, s = ss, a1 = a1s)
    return df
end

function fpart(x::AbstractFloat)::AbstractFloat
  return x - trunc(x)
end

# for legacy dfs without abilities
function df_to_spuds0(df::DataFrame)::Array{Spud}
    n = size(df)[1]
    as = Array{Spud}(undef, n)
    for i in 1:n
        as[i] = Spud(df[i, :name], df[i, :h], df[i, :f], df[i, :l], df[i, :p], df[i, :r], df[i, :s], ability_none)
    end
    return as
end

function df_to_spuds(df::DataFrame)::Array{Spud}
    n = size(df)[1]
    as = Array{Spud}(undef, n)
    for i in 1:n
        as[i] = Spud(df[i, :name], df[i, :h], df[i, :f], df[i, :l], df[i, :p], df[i, :r], df[i, :s], df[i, :a1])
    end
    return as
end



## adjust abilities

In [ ]:
nash_env_df = DataFrame(CSV.File("spudsD_mxv10_nash.csv"))
nash_env = df_to_spuds0(nash_env_df)
counts = [parse(Int, s.name[2:end]) for s in nash_env];
w = counts./sum(counts);

In [ ]:
function cost_imitative(a::Spud)
    h_inc = [4, 4, 3, 2, 2, 1, 1, 1, 0, 0] # how much H effectively increases given base H
    c =  6 + h_inc[a.h] * (a.f + a.l) + 2 * ifelse(a.l >= 8, 1, 0)
    if a.f >= 9 && a.l >= 4
        c += 3
    end
    if a.l == 10 && a.p == 10
        c += 8
    end
    return c
end

function fx_imitative(a::Spud, b::Spud)
    if b.h > a.h
        a = Spud(a.name, b.h, a.f, a.l, a.p, a.r, a.s, a.a1)
    end
    return [a, b]
end

In [ ]:
function cost_reciprocating(a::Spud)
    # how much F effectively increases given base F (times 2)
    f_inc = [8, 6, 4, 2, 1, 1, 1, 1, 1, 1]
    c = 3 + f_inc[a.f] * div(a.h + a.p + a.r + a.s, 2)
    c = c + ifelse(a.h >= 9, 1, 0) + ifelse(a.p + a.r + a.s >= 16 && a.l >= 10, 4, 0)
    return c
end

function fx_reciprocating(a::Spud, b::Spud)
    if b.f > a.f
        a = Spud(a.name, a.h, b.f, a.l, a.p, a.r, a.s, a.a1)
    end
    return [a, b]
end

In [ ]:
function cost_romantic(a::Spud)
    # how much L effectively increases given base L
    l_inc = [3, 2, 2, 1, 1, 1, 1, 0, 0, 0]
    c = 5 + l_inc[a.l] * a.h * ifelse(a.h >= 8, 2, 1)
    # nerf Spud("", 9, 1, 4, 10, 10, 10, 999)
    if a.h >= 9 && (a.p + a.r + a.s) >= 30
        c += 3
    end
    return c
end

function fx_romantic(a::Spud, b::Spud)
    if b.p > a.l
        a = Spud(a.name, a.h, a.f, b.p, a.p, a.r, a.s, a.a1)
    end
    return [a, b]
end

In [ ]:
function cost_bibliophile(a::Spud)
    # ban full p/r/s
    if a.p + a.r + a.s >= 30
        return 100
    end
    # how much L effectively increases given base L
    l_inc = [2, 2, 2, 1, 1, 1, 1, 0, 0, 0];
    # for penalizing large H
    # h_inc = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
    c = 4 + l_inc[a.l] * a.h
    return c
end

function fx_bibliophile(a::Spud, b::Spud)
    if b.r > a.l
        a = Spud(a.name, a.h, a.f, b.r, a.p, a.r, a.s, a.a1)
    end
    return [a, b]
end

In [ ]:
function cost_melodious(a::Spud)
    h_inc = [0,0,0,0,0,1,1,2,4,10]
    f_inc = [0,0,0,0,1,1,4,5,9,11]
    c = 5 + f_inc[a.f] + 2 * h_inc[a.h] + ifelse(a.r >= 10, 3, 0)
    # nerf Spud("", 3, 9, 1, 3, 2, 1, 999)
    c += ifelse(a.f >= 9 && (a.p * a.h) >= 9, 3, 0)
    return c
end

function fx_melodious(a::Spud, b::Spud)
    if b.p > b.r && b.p > b.s
        a = Spud(a.name, a.h, a.f, a.r, a.p + 3, a.r, a.s, a.a1)
    end
    return [a, b]
end

In [ ]:
function cost_iconoclast(a::Spud)
    # ban 10's
    if sum([a.h,a.f,a.l,a.p,a.r,a.s].==10) > 0
        return 100
    end
    l_inc = [0,0,0,0,0,0,0,3,7]
    s_inc = [1,2,3,3,3,3,3,3,3]
    c = 13 + (3 * l_inc[a.l] + 1) * (s_inc[a.s])
    return c
end


function fx_iconoclast(a::Spud, b::Spud)
    v = sum([b.h,b.f,b.l,b.p,b.r,b.s] .== 10)
    if v >= 3
        aa = [a.h, a.f, a.l, a.p, a.r, a.s]
        aa2 = map(x -> div(x, 4), aa)
        r = aa .+ aa2
        a = Spud(a.name, r[1], r[2], r[3], r[4], r[5], r[6], a.a1)
    end
    return [a, b]
end

In [ ]:
#df[:, :c2] = df.cost .+ (df.f .+ l_inc[df.l]) .* map(x -> div(x, 5), 4 .+ df.p .+ df.r .+ df.s) .+ 3;
function cost_intoxicating(a::Spud)
    l_inc = [0,0,0,0,0, 0,0,1,2,3]
    c = 3 + (a.f + l_inc[a.l]) * div(4 + a.p + a.r + a.s, 5)
    # nerf Spud("", 1, 4, 10, 2, 10, 3, 999)
    if a.h == 1 && a.f == 4 && a.l == 10 && a.p == 2 && a.r == 10 && a.s == 3
        c += 3
    end
    return c
end

function fx_intoxicating(a::Spud, b::Spud)
    b = Spud(b.name, b.h, b.f, b.l, b.p, b.r, b.f, b.a1)
    return [a, b]
end

In [ ]:
function cost_otaku(a::Spud)
    l_inc = [0,0,0,0,0, 0,0,0,0,8]
    c = 0 + l_inc[a.l] * div(7 + a.p + a.r + a.s, 8)
    if maximum([a.p, a.r, a.s]) == 10
        c += a.l
    end
    if c < 6
        c = 6
    end
    return c
end

function fx_otaku(a::Spud, b::Spud)
    b = Spud(b.name, b.h, b.f, b.l, b.p, b.h, b.s, b.a1)
    return [a, b]
end

In [ ]:
function cost_sincere(a::Spud)
    h_inc = [0,0,0,4,5,5,5,5,5,5]
    l_inc = [0,0,1,7,8,9,9,9,9,9]
    s_inc = [0,0,0,0,1,2,2,2,2,2]
    c = (h_inc[a.h] + s_inc[a.s]) * l_inc[a.l];
    if c < 6
        c = 6
    end
    return c
end

function fx_sincere(a::Spud, b::Spud)
    b = Spud(b.name, b.h, b.f + 2, b.f + 2, b.p, b.r, b.s, b.a1)
    return [a, b]
end

In [104]:
function cost_cowardly(a::Spud)
    if (a.f < 3) || (a.h > 8)
        return 100
    end
    c = 0
    c2 = cost(a.h + 2, a.f - 2, a.l, a.p, a.r, a.s)
    c0 = cost(a.h, a.f, a.l, a.p, a.r, a.s)
    if c2 > c0
        c = c2 - c0
    end
    f_inc = [0,0,0,0,0,2,2,3,8,10]
    l_inc = [0,0,0,0,0,1,2,3,6,7]
    c += f_inc[a.f] + l_inc[a.l]
    if c < 6
        c = 6
    end
    return c
end

function fx_cowardly(a::Spud, b::Spud)
    if (a.p < b.p) || (a.r < b.r) || (a.s < b.s)
        a = Spud(a.name, a.h + 2, a.f - 2, a.l, a.p, a.r, a.s, a.a1)
    end
    return [a, b]
end

fx_cowardly (generic function with 1 method)

In [105]:
abc = cost_cowardly
abf = fx_cowardly
ab = ability_cowardly

130

In [51]:

evs = Array{Float64}(undef, 1000000)
cs = Array{Int64}(undef, 1000000)
ffs = Array{Spud}(undef, 1000000)
ff_i = 0

ss_prob = 1.1
hrange = MNV:MXV
frange = MNV:MXV
lrange = MNV:MXV
prange = MNV:MXV
rrange = MNV:MXV
srange = MNV:MXV

for h in hrange
    if (cost(h, MNV, MNV, MNV, MNV, MNV) <= MXS)
        for f in frange
            if (cost(h, f, MNV, MNV, MNV, MNV) <= MXS)
                for l in lrange
                    if (cost(h, f, l, MNV, MNV, MNV) <= MXS)
                        for p in prange
                            if (cost(h, f, l, p, MNV, MNV) <= MXS)
                                for r in rrange
                                    if (cost(h, f, l, p, r, MNV) <= MXS)
                                        for s in srange
                                            if rand() < ss_prob && (cost(h,f,l,p,r,s) <= MXS)
                                                ff = Spud("", h,f,l,p,r,s,ability_none)
                                                ebs = [0 for i in 1:length(nash_env)]
                                                for i in 1:length(nash_env)
                                                    gg = nash_env[i]
                                                    res = abf(ff, gg)
                                                    ebs[i] = eval_battle(res[1], res[2])
                                                end
                                                ev = sum(w[ebs .== 1]) - sum(w[ebs .== -1])
                                                if abs(ev) < 0.5
                                                    ff_i += 1
                                                    ffs[ff_i] = ff
                                                    evs[ff_i] = ev
                                                    cs[ff_i] = cost(ff)
                                                end
                                            end
                                        end
                                    end
                                end                        
                            end
                        end                        
                    end
                end
            end
        end
    end
end




evs = evs[1:ff_i]
cs = cs[1:ff_i]
ffs = ffs[1:ff_i];
#maximum(evs)

In [106]:
df = spuds_to_df(ffs)
df[:, :ev] = evs
df[:, :cost] = cs
df[:, :c2] = df.cost .+ [abc(ff) for ff in ffs]
df;

In [107]:
maximum(df[df.c2 .<= 100,:ev])

0.028386574466642

In [111]:
#df2 = df[map(abs, df.ev) .< 0.02, :]
df2 = df[df.ev .> -0.1 .&& df.c2 .<= 100, :]
CSV.write("temp.csv", df2[sortperm(df2.c2), :])

"temp.csv"

In [109]:
maximum(df.ev[df.c2 .<= 100])

0.028386574466642

In [110]:
print(df[df.c2 .<= 100. .&& df.ev .> -.0, :])

2×11 DataFrame
 Row │ name    h      f      l      p      r      s      a1     ev          cost   c2    
     │ String  Int64  Int64  Int64  Int64  Int64  Int64  Int64  Float64     Int64  Int64 
─────┼───────────────────────────────────────────────────────────────────────────────────
   1 │             1      9      2      2      2      5    999  0.0283866      92    100
   2 │             1      9      2      5      2      2    999  0.00863089     92    100

In [89]:
ff = Spud("", 1,10,4,3,1,3, 999)

Spud("", 1, 10, 4, 3, 1, 3, 999)

In [90]:
wl = [eval_battle(ff, gg) for gg in nash_env];

In [78]:
sum(wl .* w)

-0.4654536756035403

In [79]:
wl2 = map(x -> eval_battle(abf(ff, x)[1], x), nash_env);

In [80]:
sum(wl2 .* w)

0.004693810896189656

In [82]:
print(spuds_to_df(nash_env[wl2 .> wl]))

30×8 DataFrame
 Row │ name       h      f      l      p      r      s      a1    
     │ String     Int64  Int64  Int64  Int64  Int64  Int64  Int64 
─────┼────────────────────────────────────────────────────────────
   1 │ c12207143      6      1     10     10     10     10    999
   2 │ c4458137       3      2     10     10     10     10    999
   3 │ c3597175       4      2     10     10     10      6    999
   4 │ c1595416       7      1     10     10      3     10    999
   5 │ c1059805       7      1     10     10     10      3    999
   6 │ c965638        7      1     10     10      9      4    999
   7 │ c597657        3     10      3      1      2      3    999
   8 │ c518213        7      1     10      3     10     10    999
   9 │ c332620        3      9      3      1      3      3    999
  10 │ c121365        7      1     10      4     10      9    999
  11 │ c102428        7      1     10     10      4      9    999
  12 │ c97280         7      1     10     10      7      6

In [83]:
print(spuds_to_df(nash_env[wl2 .< wl]))

26×8 DataFrame
 Row │ name     h      f      l      p      r      s      a1    
     │ String   Int64  Int64  Int64  Int64  Int64  Int64  Int64 
─────┼──────────────────────────────────────────────────────────
   1 │ c445008     10      4      3      3      3      1    999
   2 │ c444222     10      1      8      4      4      2    999
   3 │ c378021      2     10      5      2      1      4    999
   4 │ c227214     10      5      1      3      3      2    999
   5 │ c157012      3      9      3      4      1      2    999
   6 │ c144684     10      1      8      5      3      2    999
   7 │ c101987      9      1      9      5      3      2    999
   8 │ c100007      2     10      5      1      1      5    999
   9 │ c89887       9      6      1      2      2      2    999
  10 │ c81102       9      4      4      2      3      2    999
  11 │ c29928       9      5      2      3      2      2    999
  12 │ c25445       9      6      1      1      3      2    999
  13 │ c20678       3 

In [ ]:
evs = Array{Float64}(undef, 1000000)
cs = Array{Int64}(undef, 1000000)
ffs = Array{Spud}(undef, 1000000)
ff_i = 0

ss_prob = 1.1
hrange = MNV:MXV
frange = MNV:MXV
lrange = MNV:MXV
prange = MNV:MXV
rrange = MNV:MXV
srange = MNV:MXV

for h in hrange
    if (cost(h, MNV, MNV, MNV, MNV, MNV) <= MXS)
        for f in frange
            if (cost(h, f, MNV, MNV, MNV, MNV) <= MXS)
                for l in lrange
                    if (cost(h, f, l, MNV, MNV, MNV) <= MXS)
                        for p in prange
                            if (cost(h, f, l, p, MNV, MNV) <= MXS)
                                for r in rrange
                                    if (cost(h, f, l, p, r, MNV) <= MXS)
                                        for s in srange
                                            if rand() < ss_prob && (cost(h,f,l,p,r,s) <= MXS)
                                                ff = Spud("", h,f,l,p,r,s,ability_none)
                                                if (cost(ff) + abc(ff)) <= MXS
                                                    ebs = [0 for i in 1:length(nash_env)]
                                                    for i in 1:length(nash_env)
                                                        gg = nash_env[i]
                                                        res = abf(ff, gg)
                                                        ebs[i] = eval_battle(res[1], res[2])
                                                    end
                                                    ev = sum(w[ebs .== 1]) - sum(w[ebs .== -1])
                                                    if ev > -0.1
                                                        ff_i += 1
                                                        ffs[ff_i] = ff
                                                        evs[ff_i] = ev
                                                        cs[ff_i] = cost(ff) + abc(ff)
                                                    end
                                                end
                                            end
                                        end
                                    end
                                end                        
                            end
                        end                        
                    end
                end
            end
        end
    end
end




evs = evs[1:ff_i]
cs = cs[1:ff_i]
ffs = ffs[1:ff_i];
maximum(evs)

In [ ]:
df = spuds_to_df(ffs)
df[:, :ev] = evs
df[:, :cost] = cs
df

In [ ]:
df[:, :a1] = [ab for i in 1:size(df)[1]];

In [ ]:
df[:, :name] = ["noname" for i in 1:size(df)[1]];

In [ ]:
CSV.write("temp_cowardly.csv", df)

In [ ]:
ffs[evs .> 0]
#print(df[df.ev .> 0,:])